In [1]:
import numpy as np
from scipy.special import expit
from numpy import logaddexp
from glopt import GLMProblem,SolverSettings
from problems import build_random_logistic,build_logistic_lasso
import matplotlib.pyplot as plt
from scipy.sparse import csc_array,diags_array,csr_array,block_array
from numpy.random import default_rng
import qdldl
from util import maxnorm
from util import get_step_size

In [2]:
m = 50
n = 100
k = 10

f,A,Q,C,c,b = build_random_logistic(m = m,n = n,k = k,density=0.1, seed = 2)

rng = default_rng(10)
E = csr_array(np.vstack([rng.normal(size = (1,n))]*20))
e = np.zeros(E.shape[0])


settings = SolverSettings(tau_reg = 1e-6)
problem = GLMProblem(f,A,Q,C,c,b,E = E ,e = e,settings = settings)
x,results = problem.solve(verbose = True,mu0 = 1.) 


──────────────────────────────────────────────────────────────────────────────────────────────────────────
Variables:: 100     │ Rows in A: 50      │ Equality Constraints: 20      │ Inequality Constraints: 10     
──────────────────────────────────────────────────────────────────────────────────────────────────────────
─────────────────────────────────────────────────────────────────────────────────────────────────────────────────
│ iter │   primal   │ dual_res  │ cons_viol │ comp_res  │  KKT_res  │    mu    │   Δx    │  step   │ refine │  time  │
─────────────────────────────────────────────────────────────────────────────────────────────────────────────────
│    1 │ 3.4657e+03 │  2.74e+00 │  3.50e-15 │  3.95e+00 │  3.95e+00 │  1.0e-02 │ 2.0e+00 │ 1.0e+00 │ 1 │   0.02s │
│    2 │ 3.3322e+03 │  1.07e-01 │  3.82e-15 │  3.96e-02 │  1.07e-01 │  1.0e-04 │ 5.1e-01 │ 9.9e-01 │ 1 │   0.02s │
│    3 │ 3.3282e+03 │  1.33e-03 │  3.61e-15 │  3.97e-04 │  1.33e-03 │  1.0e-06 │ 5.0e-02 │ 9.9e-01 │ 1

/mnt/team/msca/priv/jira/MSCA-375-spxmod-optimizer/glopt/util.py:86: UserWarning: Poor linear solve: didn't reach target tolerance of 5.000e-10 in 5 steps
  warn(f"Poor linear solve: didn't reach target tolerance of {target_tol:.3e} in {num_refine} steps")


In [3]:
self = problem
tau_reg = 1e-8
x,y,s,nu = problem.initialize()
mu = 1e-14

def KKT_res(x,g,y,s,nu):
    rx = g + self.C.T@y + self.E.T@nu - self.b
    rp = self.C@x + s - self.c
    rc = y * s
    req = self.E@x - self.e
    return rx,rp,rc,req

In [4]:
for i in range(16):
    z = self.A@x
    H = self.get_H(z)

    w = np.sqrt(y/s)
    wC = self.C.multiply(w[:,None])
    gradf = self.A.T@self.f.d1f(z) + self.Q@x

    rx,rp,rc,req = self.KKT_res(x,gradf,y,s,nu)
    rc = rc - mu
    print(f"rx: {maxnorm(rx)},rp: {maxnorm(rp)}, rc: {maxnorm(rc)}, req: {maxnorm(req)}")
    rhs = np.hstack([-rx,-w*rp + (w/y) * rc,-req])

    G = block_array(
        [
            [H,     wC.T,       self.E.T],
            [wC,    -1*self.Ik, None],
            [self.E,None,       None]
        ],format = 'csc'
    )
    Gshift = G + tau_reg * self.regshift

    solver = qdldl.Solver(Gshift)

    sol = solver.solve(rhs)

    # ref = rhs - G@sol
    # sol = sol + solver.solve(ref)
    # ref = rhs - G@sol
    # sol = sol + solver.solve(ref)

    dx = sol[:self.n]
    dy = w*sol[self.n:self.n+self.k]
    ds = -rp - self.C@dx
    dnu = sol[self.n+self.k:]
    print(dnu[0])


    t = get_step_size(s,ds,y,dy,0.99)

    x = x+ t*dx
    y = y+ t*dy
    s = s+ t*ds
    nu = nu+t*dnu

    z = self.A@x
    gradf = self.A.T@self.f.d1f(z) + self.Q@x

rx: 35.79882198263401,rp: 2.6283975507171067, rc: 3.702919690066895, req: 0.0


AttributeError: 'GLMProblem' object has no attribute 'regshift'

In [119]:
rc

array([9.82943284e-18, 1.05794111e-24])

In [82]:
rp

array([ 1.71162265e-01, -8.06399392e-12])